In [ ]:
import traceback
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import time

In [ ]:
url = "https://www.ciaaw.org/historical-isotopic-abundances.htm"

In [ ]:
current_value = ''

options = Options()
driver = webdriver.Chrome(options=options)

all_list = []

try:
    # 取得先URLにアクセス
    driver.get(url)
    
    # コンテンツが描画されるまで待機
    time.sleep(5)
    
    # selectorはchromeの要素から右クリック->copy->copy selectorでげっとできる。
    selector = '#mytable > thead > tr.tablesorter-filter-row > td:nth-child(4) > input'
    element = driver.find_element_by_css_selector(selector)
    element.clear()
    
    selector = '#mytable > thead > tr.tablesorter-filter-row > td:nth-child(2) > input'
    element = driver.find_element_by_css_selector(selector)
    element.clear()
    
    element.send_keys(Keys.ENTER)
    time.sleep(10)

    # 対象を抽出
    values = driver.find_element_by_id("mytable")
    trs = values.find_elements(By.TAG_NAME, "tr")
    for i in range(1, len(trs)):
        tds = trs[i].find_elements(By.TAG_NAME, "td")
        list_tmp = []
        for j in range(len(tds)):
            print(tds[j].text)
            list_tmp.append(tds[j].text)
        
        all_list.append(list_tmp)

finally:
    driver.quit()

In [ ]:
url = "https://www.ciaaw.org/historical-atomic-weights.htm"
all_mass_list = []

options = Options()
driver = webdriver.Chrome(options=options)


try:
    # 取得先URLにアクセス
    driver.get(url)
    
    # コンテンツが描画されるまで待機
    time.sleep(5)
    
    # selectorはchromeの要素から右クリック->copy->copy selectorでげっとできる。
    selector = '#mytable > thead > tr.tablesorter-filter-row > td:nth-child(3) > input'
    element = driver.find_element_by_css_selector(selector)
    element.clear()
    
    element.send_keys(Keys.ENTER)
    time.sleep(10)

    # 対象を抽出
    values = driver.find_element_by_id("mytable")
    trs = values.find_elements(By.TAG_NAME, "tr")
    for i in range(1, len(trs)):
        tds = trs[i].find_elements(By.TAG_NAME, "td")
        list_tmp = []
        for j in range(len(tds)):
            print(tds[j].text)
            list_tmp.append(tds[j].text)
        
        all_mass_list.append(list_tmp)

finally:
    driver.quit()

In [ ]:
import copy
all_information = copy.deepcopy(all_list)

In [ ]:
all_abundance_information = copy.deepcopy(all_information)
all_mass_information = copy.deepcopy(all_mass_list)

In [ ]:
elems = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne",
            "Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca",
            "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu",
            "Zn", "Ga", "Ge", "As", "Se", "Br", "Kr", "Rb", "Sr",
            "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag",
            "Cd", "In", "Sn", "Sb", "Te", "I", "Xe", "Cs", "Ba",
            "La", "Ce", "Pr", "Nd", "Pm", "Sm", "Eu", "Gd", "Tb",
            "Dy", "Ho", "Er", "Tm", "Yb", "Lu", "Hf", "Ta", "W", "Re",
            "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", "Po", "At",
            "Rn", "Fr", "Ra", "Ac", "Th", "Pa", "U"]

elems_full_name  = ["hydrogen", "helium", "lithium", "beryllium", "boron", "carbon", "nitrogen", "oxygen", "fluorine", "neon", "sodium", "magnesium", "aluminium", "silicon", "phosphorus", "sulfur", "chlorine", "argon", "potassium", "calcium", "scandium", "titanium", "vanadium", "chromium", "manganese", "iron", "cobalt", "nickel", "copper", "zinc", "gallium", "germanium", "arsenic", "selenium", "bromine", "krypton", "rubidium", "strontium", "yttrium", "zirconium", "niobium", "molybdenum", "technetium", "ruthenium", "rhodium", "palladium", "silver", "cadmium", "indium", "tin", "antimony", "tellurium", "iodine", "xenon", "caesium", "barium", "lanthanum", "cerium", "praseodymium", "neodymium", "promethium", "samarium", "europium", "gadolinium", "terbium", "dysprosium", "holmium", "erbium", "thulium", "ytterbium", "lutetium", "hafnium", "tantalum", "tungsten", "rhenium", "osmium", "iridium", "platinum", "gold", "mercury", "thallium", "lead", "bismuth", "polonium", "astatine", "radon", "francium", "radium", "actinium", "thorium", "protactinium", "uranium"]

In [ ]:
for elem in elems[:1]:
    for entry in all_abundance_information:
        if entry[2] == elem:
            print(entry)
    
    for entry in all_mass_information:
        if entry[1] == elem:
            print(entry)

In [ ]:
import numpy as np
import re

def get_mass_information_latest(elem, neglect_data_with_range = True):
    
    list_tmp = []
    for entry in all_mass_information:
        if entry[1] == elem:
            list_tmp.append(entry)
            
    if len(list_tmp) == 0:
        return None, None
    
    list_tmp = np.array(list_tmp)
    index_sorted = np.argsort([-float(t) for t in list_tmp[:,0]])
    
    str_mass = ''
    year = None
    
    if neglect_data_with_range:
        for i in index_sorted:
            if '[' in list_tmp[i, 3]:
                continue
            else:
                str_mass = list_tmp[i, 3]
                year = int(list_tmp[i, 0])
                break
    else:
        str_mass = list_tmp[index_sorted[0], 3]
        year = int(list_tmp[index_sorted[0], 0])
        
    str_mass = "".join(str_mass.split()).split('(')[0]
    
    if '[' in str_mass:
        data = re.split('[\[\],]', str_mass)
        val_lower = float(data[1])
        val_upper = float(data[2])
        mass = [val_lower, val_upper]
    else:
        if str_mass:
            try:
                mass = float(str_mass)
            except:
                mass = None
        else:
            mass = None
        
    return mass, year


def get_abundance_information_latest(elem, neglect_data_with_range = True):
    
    dict_tmp = {}
    list_tmp = []
    
    for entry in all_abundance_information:
        if entry[2] == elem:
            list_tmp.append(entry)
            
    if len(list_tmp) == 0:
        return None, None
    
    list_tmp = np.array(list_tmp)
    index_sorted = np.argsort([-float(t) for t in list_tmp[:,0]])
    
    unique_years = list(set(list_tmp[:,0]))
    unique_years = np.sort(unique_years)[::-1]

    str_abundance = ''
    year_ret = None
    abundance = []
    
    for year in unique_years:
        list_tmp2 = []
        for entry in list_tmp:
            if entry[0] == year:
                list_tmp2.append(entry[1:])

        list_tmp2 = np.array(list_tmp2)
        index_sorted = np.argsort([int(t) for t in list_tmp2[:,0]])
        
        if neglect_data_with_range:
            if '[' in list_tmp2[0, 3]:
                continue
            else:
                for i in index_sorted:
                    strtmp = "".join(list_tmp2[i, 3].split()).split('(')[0]
                    abundance.append(float(strtmp))
                year_ret = year
                break
        else:
            for i in index_sorted:
                strtmp = "".join(list_tmp2[i, 3].split()).split('(')[0]
                if '[' in strtmp:
                    data = re.split('[\[\],]', strtmp)
                    val_lower = float(data[1])
                    val_upper = float(data[2])
                    abundance.append([val_lower, val_upper])
                else:
                    abundance.append(float(strtmp))
            year_ret = year
            break

#    print(abundance)

    return abundance, year_ret



def extract_massinfo_from_html(html):
    soup = BeautifulSoup(html, "html.parser")
    table = soup.findAll("table", {"class": "tablesorter"})[0]
    rows = table.findAll('tr')

    ret_dict = {}
    abundance_all = []
    mass_all = []
    name_all = []

    for row in rows:
        cols = row.findAll('td')
        if len(cols) == 0:
            continue

        elem_name = cols[0].get_text()
        elem_mass = "".join(cols[1].get_text().split()).split('(')[0]
        abundance = "".join(cols[2].get_text().split())

        if '[' in abundance:
                data2 = re.split('[\[\],]', abundance.strip())
                val_lower = float(data2[1])
                val_upper = float(data2[2])
                abundance_all.append([val_lower, val_upper])
        else:
            if abundance == '1':
                ratio = 1.0
            else:
                ratio = float(abundance.split('(')[0])
            abundance_all.append([ratio])

        mass_all.append(float(elem_mass))
        name_all.append(elem_name)

    ret_dict['mass'] = mass_all
    ret_dict['abundance'] = abundance_all
    ret_dict['name'] = name_all

    return ret_dict


def get_average_mass(mass_list, abundance):
    n = len(mass_list)

    if n == 0:
        return None, None

    mass_avg = 0.0

    if len(mass_list) != len(abundance):
        print("The length of mass_list and abundance differs.")
        exit(1)

    abundance_update = []

    for mass, ratio in zip(mass_list, abundance):
        if len(ratio) > 1:
            ratio_avg = np.sum(ratio) / float(len(ratio))
            abundance_update.append(ratio_avg)
        else:
            abundance_update.append(ratio[0])

    if abs(np.sum(abundance_update) - 1.0) > 0.01:
        print("The sum of the ratio %f is not 1." % np.sum(abundance_update))
        print(abundance_update)
        abundance_update[:] /= np.sum(abundance_update)
        

    for i in range(n):
        mass_avg += mass_list[i] * abundance_update[i]

    return mass_avg, abundance_update


def get_isofact(mass_list, abundance):

    mass_avg, abundance_update = get_average_mass(mass_list, abundance)

    if mass_avg is None:
        return None

    isofact = 0.0

    n = len(mass_list)

    for i in range(n):
        isofact += (1.0 - mass_list[i]/mass_avg)**2 * abundance_update[i]

    return isofact


In [ ]:
import os,sys
import urllib.request
from bs4 import BeautifulSoup

elems = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne",
            "Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca",
            "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu",
            "Zn", "Ga", "Ge", "As", "Se", "Br", "Kr", "Rb", "Sr",
            "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag",
            "Cd", "In", "Sn", "Sb", "Te", "I", "Xe", "Cs", "Ba",
            "La", "Ce", "Pr", "Nd", "Pm", "Sm", "Eu", "Gd", "Tb",
            "Dy", "Ho", "Er", "Tm", "Yb", "Lu", "Hf", "Ta", "W", "Re",
            "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", "Po", "At",
            "Rn", "Fr", "Ra", "Ac", "Th", "Pa", "U"]

elems_full_name  = ["hydrogen", "helium", "lithium", "beryllium", "boron", 
"carbon", "nitrogen", "oxygen", "fluorine", "neon", "sodium", "magnesium", "aluminium", "silicon", "phosphorus", "sulfur", "chlorine", "argon", "potassium", "calcium", "scandium", "titanium", "vanadium", "chromium", "manganese", "iron", "cobalt", "nickel", "copper", "zinc", "gallium", "germanium", "arsenic", "selenium", "bromine", "krypton", "rubidium", "strontium", "yttrium", "zirconium", "niobium", "molybdenum", "technetium", "ruthenium", "rhodium", "palladium", "silver", "cadmium", "indium", "tin", "antimony", "tellurium", "iodine", "xenon", "caesium", "barium", "lanthanum", "cerium", "praseodymium", "neodymium", "promethium", "samarium", "europium", "gadolinium", "terbium", "dysprosium", "holmium", "erbium", "thulium", "ytterbium", "lutetium", "hafnium", "tantalum", "tungsten", "rhenium", "osmium", "iridium", "platinum", "gold", "mercury", "thallium", "lead", "bismuth", "polonium", "astatine", "radon", "francium", "radium", "actinium", "thorium", "protactinium", "uranium"]

#print(len(elems))
#print(len(elems_full_name))

download_from_web = False

for elem, element_name in zip(elems, elems_full_name):
    
    #print(elem)
    mass, year1 = get_mass_information_latest(elem, neglect_data_with_range = True)
    abundance, year2 = get_abundance_information_latest(elem, neglect_data_with_range = True)
    
    try:
        if download_from_web:
            url = "https://www.ciaaw.org/%s.htm" % element_name
            urllib.request.urlretrieve(url, '%s.htm' % element_name)
            time.sleep(2)

        html = open('%s.htm' % element_name, 'r')
        dict = extract_massinfo_from_html(html)

    except:
        dict = None

    if dict:
#        print(dict['abundance'])
#        print(abundance)
        mass_avg, _ = get_average_mass(dict['mass'], dict['abundance'])
        mass_avg2, _ = get_average_mass(dict['mass'], [[i] for i in abundance])

        isofact = get_isofact(dict['mass'], dict['abundance'])
        isofact2 = get_isofact(dict['mass'], [[i] for i in abundance])

        print("%3s %.5f %.5f (%4d) %.5f (%4d): %.5f: %f %f" % 
              (elem, mass_avg, mass_avg2, int(year2), mass, int(year1), mass_avg2 - mass,
              isofact, isofact2))
        
    else:
        print(elem, None)

In [ ]:
import os,sys
import urllib.request
from bs4 import BeautifulSoup

elems = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne",
            "Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca",
            "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu",
            "Zn", "Ga", "Ge", "As", "Se", "Br", "Kr", "Rb", "Sr",
            "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag",
            "Cd", "In", "Sn", "Sb", "Te", "I", "Xe", "Cs", "Ba",
            "La", "Ce", "Pr", "Nd", "Pm", "Sm", "Eu", "Gd", "Tb",
            "Dy", "Ho", "Er", "Tm", "Yb", "Lu", "Hf", "Ta", "W", "Re",
            "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", "Po", "At",
            "Rn", "Fr", "Ra", "Ac", "Th", "Pa", "U"]

elems_full_name  = ["hydrogen", "helium", "lithium", "beryllium", "boron", 
"carbon", "nitrogen", "oxygen", "fluorine", "neon", "sodium", "magnesium", "aluminium", "silicon", "phosphorus", "sulfur", "chlorine", "argon", "potassium", "calcium", "scandium", "titanium", "vanadium", "chromium", "manganese", "iron", "cobalt", "nickel", "copper", "zinc", "gallium", "germanium", "arsenic", "selenium", "bromine", "krypton", "rubidium", "strontium", "yttrium", "zirconium", "niobium", "molybdenum", "technetium", "ruthenium", "rhodium", "palladium", "silver", "cadmium", "indium", "tin", "antimony", "tellurium", "iodine", "xenon", "caesium", "barium", "lanthanum", "cerium", "praseodymium", "neodymium", "promethium", "samarium", "europium", "gadolinium", "terbium", "dysprosium", "holmium", "erbium", "thulium", "ytterbium", "lutetium", "hafnium", "tantalum", "tungsten", "rhenium", "osmium", "iridium", "platinum", "gold", "mercury", "thallium", "lead", "bismuth", "polonium", "astatine", "radon", "francium", "radium", "actinium", "thorium", "protactinium", "uranium"]

#print(len(elems))
#print(len(elems_full_name))

download_from_web = False

mass_to_use = []
isofact_to_use = []

for elem, element_name in zip(elems, elems_full_name):
    
    # standard mass from the relatively old database without information of deviation range
    mass, year1 = get_mass_information_latest(elem, neglect_data_with_range = True)
    
    # abundance from the relatively old database without information of deviation range
    abundance, year2 = get_abundance_information_latest(elem, neglect_data_with_range = True)
    
    #Only dict['mass'] will be used for production data
    try:
        if download_from_web:
            url = "https://www.ciaaw.org/%s.htm" % element_name
            urllib.request.urlretrieve(url, '%s.htm' % element_name)
            time.sleep(2)

        html = open('%s.htm' % element_name, 'r')
        dict = extract_massinfo_from_html(html)

    except:
        dict = None

    if dict:
        #mass_avg, _ = get_average_mass(dict['mass'], dict['abundance'])
        mass_avg2, _ = get_average_mass(dict['mass'], [[i] for i in abundance])

        #isofact = get_isofact(dict['mass'], dict['abundance'])
        isofact2 = get_isofact(dict['mass'], [[i] for i in abundance])
        
        mass_to_use.append(mass)
        isofact_to_use.append(isofact2)
        
    else:
        mass_to_use.append(-1)
        isofact_to_use.append(isofact2)
        

In [ ]:
for entry in mass_to_use:
    print("%s," %  entry, end = '')

In [ ]:
olddata = [1.007941, 4.002602, 6.940037, 9.012183, 10.811028, 12.010736, 14.006703, 15.999405, 18.998403,
            20.180046,
            22.989769, 24.305052, 26.981539, 28.085499, 30.973762, 32.064787, 35.452938, 39.947799, 39.098301,
            40.078023, 44.955908, 47.866745, 50.941465, 51.996132, 54.938044, 55.845144, 58.933194, 58.693347,
            63.546040, 65.377783, 69.723066, 72.627550, 74.921595, 78.959389, 79.903528, 83.798000, 85.467664,
            87.616644, 88.905840, 91.223642, 92.906373, 95.959789, -1, 101.064940, 102.905498, 106.415328,
            107.868150, 112.411558, 114.818087, 118.710113, 121.759784, 127.603126, 126.904472, 131.292761,
            132.905452,
            137.326892, 138.905469, 140.115731, 140.907658, 144.241596, -1, 150.366356, 151.964378, 157.252131,
            158.925355, 162.499473, 164.930329, 167.259083, 168.934218, 173.054150, 174.966815, 178.484979,
            180.947876,
            183.841778, 186.206705, 190.224860, 192.216052, 195.084457, 196.966569, 200.599167, 204.383413,
            207.216908,
            208.980399, -1, -1, -1, -1, -1, -1, 232.038056, 231.035884, 238.028910, -1, -1]

for new, old in zip(mass_to_use, olddata):
    print(new, old, new-old)

In [ ]:
for isofact, mass in zip(isofact_to_use, mass_to_use):
    if mass == -1:
        print("%s," %  -1, end = '')
    else:
        print("%s," %  isofact, end = '')


In [ ]:
olddata_iso = [           0.00011460742533846188, 8.141017510851699e-08, 0.001458820047621947, 0.0, 0.0013539153245004177,
            7.387230454537788e-05, 1.8376657914224594e-05, 3.3588025065260784e-05, 0.0, 0.0008278312160539562, 0.0,
            0.0007398827105109379, 0.0, 0.00020070043953754622, 0.0, 0.00016512811739588583, 0.0005827012790438193,
            3.4803742597617525e-05, 0.0001640001019257936, 0.00029756377957496945, 0.0, 0.00028645556480110174,
            9.548320927761995e-07, 0.0001328704171052577, 0.0, 8.244411853712163e-05, 0.0, 0.00043070442089852476,
            0.00021093312802220393, 0.0005931533235650079, 0.00019712731536263687, 0.000586966208421238, 0.0,
            0.0004627901461335774, 0.00015627716697599108, 0.00024880742737704476, 0.00010969638987469167,
            6.099700015374489e-05, 0.0, 0.00034262903024295327, 0.0, 0.0005961083777486782, -1,
            0.0004066661504740231,
            0.0, 0.0003094784411091063, 8.579847704787673e-05, 0.0002716036180261363, 1.245588189674909e-05,
            0.0003340852797872776, 6.607553852631361e-05, 0.0002839333030058624, 0.0, 0.0002675566535085368, 0.0,
            6.237013178021676e-05, 4.5917491111023726e-08, 2.2495590932891925e-05, 0.0, 0.00023237187990100274, -1,
            0.000334685954430736, 4.3279441126609935e-05, 0.0001276749037273739, 0.0, 5.198070714285335e-05, 0.0,
            7.23248017569606e-05, 0.0, 8.556028002982757e-05, 8.300794202558322e-07, 5.253850496170609e-05,
            3.6715121208243084e-09, 6.966807117351981e-05, 2.7084982818795603e-05, 7.452354225251159e-05,
            2.5378700157091918e-05, 3.4285145177491544e-05, 0.0, 6.525193204276608e-05, 1.9964351041965618e-05,
            1.9437780365209887e-05, 0.0, -1, -1, -1, -1, -1, -1, 0.0, 0.0, 1.1564592331193284e-06, -1, -1]

for isofact, mass, old in zip(isofact_to_use, mass_to_use, olddata_iso):
    if mass == -1:
        print(-1, old, -1-old)
    else:
        print(isofact, old, isofact-old)